In [ ]:
!pip install boto3

In [2]:
# Import S3 connectors librairies
#from awsPy.aws_authorization import aws_connector
import importlib.util
spec = importlib.util.spec_from_file_location("aws_connector.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_authorization/aws_connector.py")
aws_connector = importlib.util.module_from_spec(spec)
spec.loader.exec_module(aws_connector)

#from awsPy.aws_s3 import service_s3
import importlib.util
spec = importlib.util.spec_from_file_location("service_s3.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_s3/service_s3.py")
service_s3 = importlib.util.module_from_spec(spec)
spec.loader.exec_module(service_s3)

In [3]:
# Connect to S3
import os
from pathlib import Path

bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

con = aws_connector.aws_instantiate(credential = path_cred,
                                        region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = 'calfdata')

In [ ]:
# List files to send and compute each destination
prep_path='data/prep'
list_files=[]
for root,dirs,files in os.walk(prep_path):
    for file in files:
          if file.endswith(('.csv','.zip')):
                list_files.append((file, u.calc_dest(filename=file)))
list_files[:5]

In [ ]:
# Send files to s3
for (file_to_upload,destination_in_s3) in list_files:
    # if file exist in S3?
    s3.upload_file(file_to_upload, destination_in_s3)

In [5]:
client["resource"]

s3.ServiceResource()

In [ ]:
(file_to_upload,destination_in_s3)=list_files[0]

full_path="{}/{}".format(destination_in_s3,file_to_upload)
# check file existence in S3
client["resource"] # is equivalent to :
s3 = boto3.resource('s3')

try:
    s3.Object(bucket, 'dootdoot.jpg').load()
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        # The object does not exist.
        ...
    else:
        # Something else has gone wrong.
        raise
else:
    # The object does exist.
    ...

In [ ]:
# A ajouter dans la classe service_s3

def list_files_s3(self,path):
        """
        List all csv files found in S3 inside a certain path and saves it to a csv.

        args:
        path: Any file path from the INPI source where the files should be taken from
        #Ex : 'INPI/TC_1/Flux/2018'
        #Ex : 'INPI/TC_1/Flux/2018/01'

        returns:
        Nothing

        """
        import boto3, csv
        
        # Get existing files : S3 list all keys inside path and write in csv
        s3 = boto3.resource('s3')
        prefix_ = "{}/".format(path)#'INPI/TC_1/Flux/2018/'# syntax 'prefix/'

        with open('s3_file_list.csv','w') as newFile:
            newFileWriter = csv.writer(newFile)
            fnames = ['csv_filename', 's3_path','s3_fullname']
            writer = csv.DictWriter(newFile, fieldnames=fnames) 
            writer.writeheader()

            for bucket in s3.buckets.all():
                for obj in bucket.objects.filter(Prefix=prefix_):
                    filepath = obj.key
                    s = re.compile(r'\/+').split(str(filepath))
                    filename = s[len(s)-1]
                    split=filepath.split("/")
                    s3_path = filepath[0:len(filepath)-len(filename)]

                    writer.writerow({'csv_filename': filename,'s3_path' : s3_path,'s3_fullname' : filepath})
